 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8887/notebooks/input_queue.ipynb#Old-stuff" data-toc-modified-id="Old-stuff-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Old stuff</a></span></li></ul></div>

In [20]:
# Imports 
from importlib import reload
import utils; reload(utils); from utils import *
import data_generator; reload(data_generator); from data_generator import *
import models; reload(models); from models import *

In [2]:
# Limit Tensorflow GPU memory usage. 
# Note that it's not possible to change the allocation or release memory again.
config = tf.ConfigProto()
config.gpu_options.allow_growth = True   # Allocates as much memory as needed.
keras.backend.tensorflow_backend.set_session(tf.Session(config=config))

In [4]:
data_dir = '/local/S.Rasp/cbrain_data/' # Full dataset
%ls $data_dir

train_year0_features.nc  train_year0_targets.nc   valid_year1_targets.nc
train_year0_norm.nc      valid_year1_features.nc


In [5]:
feature_fn = 'train_year0_features.nc'
target_fn = 'train_year0_targets.nc'

In [6]:
batch_size = 1024

In [21]:
gen_obj = DataGenerator(data_dir, feature_fn, target_fn, batch_size)

Generator will have 143515648 samples in 140152 batches


In [22]:
gen = gen_obj.return_generator()

In [27]:
x, y = next(gen)

In [28]:
x.shape, y.shape

((1024, 87), (1024, 86))

## Old stuff

In [6]:
valid_fn = 'SPCAM_outputs_valid_by_lon_flat.nc'

In [7]:
train_fn = 'SPCAM_outputs_train_by_lon_flat.nc'

In [8]:
import timeit

In [20]:
f_lf = h5py.File(data_dir + train_fn, 'r')

In [21]:
f_ll = h5py.File(data_dir_ll + train_fn, 'r')

In [22]:
f_lf['QRL'].shape

(21, 27613440)

In [23]:
f_ll['QRL'].shape

(27613440, 21)

In [24]:
batch_size = 1024
n_samples = f_lf['LAT'].shape[0]
n_batches = int(n_samples / batch_size)
print('n_batches', n_batches)
# Create ID list
idxs = np.arange(0, n_samples, batch_size)
np.random.shuffle(idxs)

n_batches 26966


In [25]:
t1 = timeit.default_timer()
for i in range(401):
    if i % 200 == 0: print(i)
    tmp = f_lf['TAP'][:, idxs[i]:idxs[i] + batch_size]
t2 = timeit.default_timer()
print(t2 - t1)

0
200
400
69.08760366500064


In [26]:
t1 = timeit.default_timer()
for i in range(401):
    if i % 200 == 0: print(i)
    tmp = f_ll['TAP'][:, idxs[i]:idxs[i] + batch_size]
t2 = timeit.default_timer()
print(t2 - t1)

0
200
400
0.19268482200277504


In [27]:
t1 = timeit.default_timer()
for i in range(401):
    if i % 200 == 0: print(i)
    tmp = f_lf['TAP'][:, idxs[i]:idxs[i] + batch_size]
t2 = timeit.default_timer()
print(t2 - t1)

0
200
400
0.34314780199929373


In [28]:
t1 = timeit.default_timer()
for i in range(401):
    if i % 200 == 0: print(i)
    tmp = f_ll['TAP'][:, idxs[i]:idxs[i] + batch_size]
t2 = timeit.default_timer()
print(t2 - t1)

0
200
400
0.1816067550025764


In [29]:
t1 = timeit.default_timer()
for i in range(401):
    if i % 200 == 0: print(i)
    tmp = f_lf['dQdt_adiabatic'][:, idxs[i]:idxs[i] + batch_size]
t2 = timeit.default_timer()
print(t2 - t1)

0
200
400
72.48782062700047


In [30]:
t1 = timeit.default_timer()
for i in range(401):
    if i % 200 == 0: print(i)
    tmp = f_ll['dQdt_adiabatic'][:, idxs[i]:idxs[i] + batch_size]
t2 = timeit.default_timer()
print(t2 - t1)

0
200
400
0.18379856800311245


In [31]:
f_ll.close(), f_lf.close()

(None, None)

In [62]:
import data_generator; reload(data_generator); from data_generator import *

In [63]:
gen1 = data_generator1(data_dir_ll, 'SPCAM_outputs_valid_by_lon_flat.nc', 'SPCAM_mean_detailed.nc',
                        'SPCAM_std_detailed.nc', feature_vars.keys(),
                        target_names=target_vars.keys(), shuffle=True)

In [67]:
%timeit next(gen1)

209 ms ± 23.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [80]:
f_ll = h5py.File(data_dir_ll + train_fn, 'r')

In [81]:
mean_file = h5py.File(data_dir_ll + 'SPCAM_mean_detailed.nc', 'r')
std_file = h5py.File(data_dir_ll + 'SPCAM_std_detailed.nc', 'r')

In [102]:
i_test = 0

In [103]:
def get_next(out_file, idxs, feature_names, target_names):
    global i_test
    batch_idx = idxs[i_test]
    i_test += 1
    # x
    f_list = []
    for v in feature_names:
        if out_file[v].ndim == 2:
            f = out_file[v][batch_idx:batch_idx + batch_size, :]
        elif out_file[v].ndim == 1:
            f = np.atleast_2d(
                out_file[v][batch_idx:batch_idx + batch_size]
            ).T
        else:
            raise ValueError('Wrong feature dimensions.')
        # normalize
        f = (f - mean_file[v].value) / std_file[v].value
        f_list.append(f)
    x = np.concatenate(f_list, axis=1)

    # y
    t_list = []
    for v in target_names:
        if out_file[v].ndim == 2:
            t = out_file[v][batch_idx:batch_idx + batch_size, :]
        elif out_file[v].ndim == 1:
            t = np.atleast_2d(
                out_file[v][batch_idx:batch_idx + batch_size]
            ).T
        t = t * conversion_dict[v]
        t_list.append(t)
    y = np.concatenate(t_list, axis=1)
    return (x, y)

In [107]:
%timeit get_next(f_ll, idxs, feature_vars.keys(), target_vars.keys())

292 ms ± 64.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
rg = nc.Dataset(data_dir_ll + 'SPCAM_outputs_valid_prestack.nc')

In [10]:
rg

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    log: 
    Time: 2017-11-24T11:31:05

    Executed command:

    python pre_stack_and_norm.py --in_fn /project/meteo/w2w/A6/S.Rasp/SP-CAM/preprocessed_data/SOLIN_lev_last/SPCAM_outputs_valid_by_lon_flat.nc --out_fn /project/meteo/w2w/A6/S.Rasp/SP-CAM/preprocessed_data/SOLIN_lev_last/SPCAM_outputs_valid_prestack.nc --mean_fn /project/meteo/w2w/A6/S.Rasp/SP-CAM/preprocessed_data/SOLIN_lev_last/SPCAM_mean_detailed.nc --std_fn /project/meteo/w2w/A6/S.Rasp/SP-CAM/preprocessed_data/SOLIN_lev_last/SPCAM_std_detailed.nc --config_file None

    In directory: /home/s/S.Rasp/repositories/CBRAIN/data_processing

    Git hash: 5c2018819acf3f0f3ba5bd7627440b3eeaed7e7c

        
    dimensions(sizes): sample(7038720), features(87), targets(86)
    variables(dimensions): float32 features(sample,features), float32 targets(sample,targets)
    groups: 

In [11]:
rg.variables['features'][:512, :]

array([[ 1.79428971,  1.92524457,  1.75502491, ..., -0.32012931,
        -0.73528463, -0.7708559 ],
       [ 1.79278553,  1.92402673,  1.75302529, ..., -0.31798324,
        -0.71717626, -0.7708559 ],
       [ 1.79116273,  1.92272329,  1.75093138, ..., -0.31662139,
        -0.7150324 , -0.7708559 ],
       ..., 
       [ 0.68742102,  0.7061252 ,  0.68143803, ..., -0.60456729,
        -0.62642539, -0.7708559 ],
       [ 0.73454267,  0.78132176,  0.75394708, ..., -0.34342924,
        -0.27924192, -0.7708559 ],
       [ 0.78267992,  0.85520601,  0.83647603, ...,  0.19240347,
         0.17712346, -0.7708559 ]], dtype=float32)

In [13]:
rg.close()

In [9]:
f_pre = h5py.File(data_dir_ll + 'SPCAM_outputs_valid_prestack.nc', 'r')

In [10]:
f_pre['features'].shape

(7038720, 87)

In [11]:
batch_size = 1024
n_samples = f_pre['features'].shape[0]
n_batches = int(n_samples / batch_size)
print('n_batches', n_batches)
# Create ID list
idxs = np.arange(0, n_samples, batch_size)
np.random.shuffle(idxs)

n_batches 6873


In [12]:
i_test = 0

In [13]:
def get_next2(out_file, idxs):
    global i_test
    batch_idx = idxs[i_test]
    i_test += 1
    if i_test >= n_batches:
        i_test = 0
    x = out_file['features'][batch_idx:batch_idx + batch_size, :]
    y = out_file['targets'][batch_idx:batch_idx + batch_size, :]
    return (x, y)

In [14]:
%timeit get_next2(f_pre, idxs)

823 µs ± 3.53 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [15]:
f_pre2 = h5py.File(data_dir_ll + 'SPCAM_outputs_train_prestack.nc', 'r')

In [17]:
%timeit get_next2(f_pre2, idxs)

12.4 ms ± 1.72 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
f_pre3 = h5py.File('/local/S.Rasp/tmp/' + 'SPCAM_outputs_train_prestack.nc', 'r')

In [19]:
%timeit get_next2(f_pre3, idxs)

5.04 ms ± 383 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [112]:
import data_generator; reload(data_generator); from data_generator import *


In [116]:
gen2 = data_generator2(data_dir_ll, 'SPCAM_outputs_valid_prestack.nc', shuffle=True, 
                       batch_size=512)

In [117]:
next(gen2)

KeyboardInterrupt: 

In [114]:
%timeit next(gen2)

KeyboardInterrupt: 

In [108]:
import netCDF4 as nc

In [110]:
rg = nc.Dataset(data_dir + train_fn)

In [111]:
rg.dimensions['sample'].size

27613440

In [55]:
def prof():
    for i in range(400):
        next(gen1)

In [56]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [57]:
%lprun -f prof()

/home/s/S.Rasp/.conda/envs/cbrain_gpu/lib/python3.6/site-packages/line_profiler.py:294: UserWarning: Could not extract a code object for the object None
  profile = LineProfiler(*funcs)


In [16]:
valid_set_lon.features[0, :5]

array([ 1.79428971,  1.92524457,  1.75502491,  1.05126357,  0.12397254], dtype=float32)

In [26]:
x1, y1 = next(gen1)

7038720 13747


In [28]:
x1[0, :5]

array([ 1.79428971,  1.92524457,  1.75502491,  1.05126357,  0.12397254], dtype=float32)

In [7]:
train_generator = oldDataGenerator(data_dir_old, 'SPCAM_outputs.nc', 512, feature_vars.keys())

In [8]:
x, y = next(train_generator.generate())

In [9]:
x.shape, y.shape

((512, 150), (512, 42))

In [10]:
%timeit [next(train_generator.generate()) for i in range(1000)]

9.8 s ± 23 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
train_generator = oldDataGenerator(data_dir_old, 'SPCAM_outputs.nc', 512, feature_vars.keys())

In [12]:
%timeit [next(train_generator.generate()) for i in range(1000)]

9.58 s ± 70.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
train_generator = test_data_generator(data_dir_old, 'SPCAM_outputs.nc', 512, feature_vars.keys())

In [6]:
tr2 = oldDataGenerator(data_dir_old, 'SPCAM_outputs.nc', 512, feature_vars.keys())

In [39]:
g = train_generator.generate()

AttributeError: 'threadsafe_iter' object has no attribute 'generate'

In [7]:
model = Sequential([
    Dense(42, input_shape=(150,))
])
model.compile(Adam(), loss='mae')

In [8]:
model.fit_generator(tr2.generate(), 9024)

Epoch 1/1
9024/9024 [==============================] - 151s - loss: 0.2168   


In [51]:
model.fit_generator(train_generator, 9024, workers=8)

Epoch 1/1
1885/9024 [=====>........................] - ETA: 167s - loss: 0.1580

KeyboardInterrupt: 

In [15]:
model.fit_generator(train_generator.generate(), train_generator.n_batches)

Epoch 1/1
9024/9024 [==============================] - 149s - loss: 0.2218   - ETA: 0s - los - ETA: 0s - loss: 


In [18]:
model.fit_generator(train_generator.generate(), train_generator.n_batches, workers=4)

Epoch 1/1


Exception in thread Thread-8:
Traceback (most recent call last):
  File "/home/s/S.Rasp/.conda/envs/cbrain_gpu/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/s/S.Rasp/.conda/envs/cbrain_gpu/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/s/S.Rasp/.conda/envs/cbrain_gpu/lib/python3.6/site-packages/keras/engine/training.py", line 612, in data_generator_task
    generator_output = next(self._generator)
ValueError: generator already executing



ValueError: output of generator should be a tuple `(x, y, sample_weight)` or `(x, y)`. Found: None